In [1]:
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from torchvision.transforms import Compose, ToTensor
from sklearn.preprocessing import LabelEncoder

In [2]:
# Define your dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = "InfraredSolarModules/" + self.dataframe.iloc[idx]['image_filepath']
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
        if self.transform:
            img = self.transform(img)
        label = int(self.dataframe.iloc[idx]['class_code'])
        return img, label

In [3]:
# Read data
DataSolarModules = pd.read_json('InfraredSolarModules/module_metadata.json').transpose().sort_index()
Classes = DataSolarModules['anomaly_class'].unique()
class_to_number = dict(enumerate(Classes.flatten(), 0))
class_to_number = {v: k for k, v in class_to_number.items()}

In [4]:
def map_to_class(value):
    return class_to_number.get(value, 'Unknown')

DataSolarModules['class_code'] = DataSolarModules['anomaly_class'].apply(map_to_class)

In [5]:
# Split data indices
datos = list(range(0,len(DataSolarModules)))
np.random.shuffle(datos)
data_train_idx, data_test_idx = train_test_split(datos, test_size=0.2)
data_train_idx, data_val_idx = train_test_split(data_train_idx, test_size=0.25)

In [6]:
# Get corresponding data using indices
data_train = DataSolarModules.iloc[data_train_idx]
data_val = DataSolarModules.iloc[data_val_idx]
data_test = DataSolarModules.iloc[data_test_idx]

In [7]:
# Prepare datasets and data loaders
transform = Compose([ToTensor()])
train_dataset = CustomDataset(data_train, transform=transform)
val_dataset = CustomDataset(data_val, transform=transform)
test_dataset = CustomDataset(data_test, transform=transform)

In [8]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [9]:
# Define CNN architecture
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.ReLU()(self.conv2(x))
        x = nn.ReLU()(self.conv3(x))
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [10]:
# Train CNN and Decision Tree jointly
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_model = CNN(num_classes=len(Classes)).to(device)
criterion = nn.CrossEntropyLoss()
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)
decision_tree = DecisionTreeClassifier()

In [11]:
for epoch in range(num_epochs):
    cnn_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # No need to convert labels to tensor
        cnn_optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        cnn_optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f"CNN Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")



    # Update decision tree with new features
    cnn_model.eval()
    train_features = []
    train_labels = []
    with torch.no_grad():
        for images, labels in train_loader:
            images = images.to(device)
            output = cnn_model(images)
            train_features.append(output.cpu().numpy())
            train_labels.append(labels.numpy())
    train_features = np.concatenate(train_features)
    train_labels = np.concatenate(train_labels)
    decision_tree.fit(train_features, train_labels)

CNN Epoch [1/10], Loss: 1.8241
CNN Epoch [2/10], Loss: 1.7508
CNN Epoch [3/10], Loss: 1.7414
CNN Epoch [4/10], Loss: 1.7352
CNN Epoch [5/10], Loss: 1.7304
CNN Epoch [6/10], Loss: 1.7256
CNN Epoch [7/10], Loss: 1.7195
CNN Epoch [8/10], Loss: 1.7123
CNN Epoch [9/10], Loss: 1.7095
CNN Epoch [10/10], Loss: 1.6918


In [12]:
# Evaluate Decision Tree
test_features = []
test_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        output = cnn_model(images)
        test_features.append(output.cpu().numpy())
        test_labels.append(labels.numpy())
test_features = np.concatenate(test_features)
test_labels = np.concatenate(test_labels)
accuracy = decision_tree.score(test_features, test_labels)
print(f"Decision Tree Accuracy on test set: {accuracy * 100:.2f}%")

Decision Tree Accuracy on test set: 37.67%
